# Keras MNIST mlp

Dimension: 784x10

In [2]:
import keras as ks

Using TensorFlow backend.


In [19]:
import numpy as np
from functools import reduce

### Load data

In [4]:
from mnist import MNIST
#you can find python-mnist source code on https://github.com/sorki/python-mnist

datahandler = MNIST('../mnist_data_loader') #change for data path
train_data = datahandler.load_training()

In [5]:
image_array = np.asarray(train_data[0])

In [6]:
label_list = []
for i in train_data[1]:
    l = [0]*10
    l[i] = 1
    label_list.append(l)

label_array = np.asarray(label_list)

In [7]:
#set data slices
train_finish = 50000
eval_finish = 55000
predict_start = 55000

### Create and train model

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.optimizers import SGD

In [9]:
model = Sequential([
    Dense(10, input_dim=784, activation = 'sigmoid'),
    Dense(10, activation = 'softmax'),
])

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [12]:
hist = model.fit(image_array[:train_finish], 
                 label_array[:train_finish],
                 validation_data = (image_array[train_finish:eval_finish],label_array[train_finish:eval_finish]),
                 nb_epoch=10, 
                 batch_size=100)

Train on 50000 samples, validate on 5000 samples
Epoch 1/10
50000/50000 [==============================] - 2s - loss: 1.8366 - acc: 0.4843 - val_loss: 1.5479 - val_acc: 0.6740
Epoch 2/10
50000/50000 [==============================] - 1s - loss: 1.4101 - acc: 0.7099 - val_loss: 1.2930 - val_acc: 0.7426
Epoch 3/10
50000/50000 [==============================] - 2s - loss: 1.1806 - acc: 0.7704 - val_loss: 1.0991 - val_acc: 0.7892
Epoch 4/10
50000/50000 [==============================] - 1s - loss: 1.0294 - acc: 0.8003 - val_loss: 0.9828 - val_acc: 0.8004
Epoch 5/10
50000/50000 [==============================] - 2s - loss: 0.9202 - acc: 0.8114 - val_loss: 0.8793 - val_acc: 0.8154
Epoch 6/10
50000/50000 [==============================] - 2s - loss: 0.8456 - acc: 0.8177 - val_loss: 0.8041 - val_acc: 0.8190
Epoch 7/10
50000/50000 [==============================] - 2s - loss: 0.7812 - acc: 0.8281 - val_loss: 0.7525 - val_acc: 0.8310
Epoch 8/10
50000/50000 [==============================] - 2s -

### Make predictions

In [14]:
p = model.predict([image_array[predict_start:]])

In [16]:
len(p) #check number of predictions

5000

# Metrics

### Confusion matrix
Useful for metric calculation

In [20]:
conf_mtx = np.zeros([10, 10])
for j in range(0, len(p)):
    prediction = p[j]
    max_pred = reduce(lambda x, y: (x if prediction[x] > prediction[y] else y), range(0,10))
    actual_label = list(filter(lambda i: label_array[predict_start + j][i], range(0,10)))[0]
    conf_mtx[max_pred][actual_label] += 1
print(conf_mtx)

[[ 459.    1.   21.   12.    1.   14.    6.    5.    2.    3.]
 [   0.  513.    4.    5.    1.    1.    3.    7.   17.    3.]
 [   4.    2.  408.   13.    0.    5.    2.    0.    6.    4.]
 [   6.    0.    5.  419.    0.   35.    0.    2.   23.    6.]
 [   0.    3.    9.    5.  478.    8.   16.    8.   11.   60.]
 [   2.    2.    1.   18.    0.  331.    1.    0.    8.    7.]
 [   4.    1.   11.    3.    8.   21.  453.    0.    2.    4.]
 [   9.    2.   15.    3.    1.   11.    0.  512.   15.   19.]
 [   4.    6.   19.   27.    2.   27.    1.    1.  407.    2.]
 [   1.    0.    0.    4.    8.    5.    0.   28.    3.  375.]]


### Accuracy

In [21]:
acc = (sum(conf_mtx[i][i] for i in range(0, 10))/5000)

In [22]:
print('Accuracy: {:.5f}'.format(acc))

Accuracy: 0.87100


### Recall

In [23]:
recall = lambda i: (conf_mtx[i][i]/sum(conf_mtx[j][i] for j in range(0,10)))

In [24]:
recall_sum = 0
for i in range(0,10):
    rcl = recall(i)
    recall_sum += rcl
    print('Recall {}: {:.5f}'.format(i, rcl))
print()
print('Recall mean: {:.5f}'.format(recall_sum/10))

Recall 0: 0.93865
Recall 1: 0.96792
Recall 2: 0.82759
Recall 3: 0.82318
Recall 4: 0.95792
Recall 5: 0.72271
Recall 6: 0.93983
Recall 7: 0.90941
Recall 8: 0.82389
Recall 9: 0.77640

Recall mean: 0.86875


### Precision

In [25]:
precision = lambda i: (conf_mtx[i][i]/sum(conf_mtx[i][j] for j in range(0,10)))

In [26]:
precision_sum = 0
for i in range(0,10):
    label_precision = precision(i)
    precision_sum += label_precision
    print('Precision {}: {:.5f}'.format(i, label_precision))
print()
print('Precision mean: {:.5f}'.format(precision_sum/10))

Precision 0: 0.87595
Precision 1: 0.92599
Precision 2: 0.91892
Precision 3: 0.84476
Precision 4: 0.79933
Precision 5: 0.89459
Precision 6: 0.89349
Precision 7: 0.87223
Precision 8: 0.82056
Precision 9: 0.88443

Precision mean: 0.87303
